### Practice 3 - Question 1

`Instructions`
- From the provided avro files at below HDFS location
	- /user/hive/warehouse/orders
	- /user/hive/warehouse/customers
- Find out customers who have not placed any order in March 2013.


`Output Requirement`
- Output should be stored in json format at below HDFS location /user/cca/practice3/ques1
- Output should have two fields customer_fname:customer_lname and customer_city:customer_state.

In [ ]:
#load data into HDFS
sqoop import  --connect "jdbc:mysql://localhost/retail_db" --username root --password admin --table orders  --warehouse-dir "/user/hive/warehouse/" --compress --compression-codec snappy --as-avrodatafile
sqoop import  --connect "jdbc:mysql://localhost/retail_db" --username root --password admin --table customers  --warehouse-dir "/user/hive/warehouse/" --compress --compression-codec snappy --as-avrodatafile 

#load dataset
orders = spark.read.option("sep","\t").format("csv").load("/user/cloudera/practice2/problem3/orders")
orderItems = spark.read.option("sep","\t").format("csv").load("/user/cloudera/practice2/problem3/order_items")
customers = spark.read.option("sep","\t").format("csv").load("/user/cloudera/practice2/problem3/customers")

#rename column names
orders = orders.selectExpr("_c0 as order_id", "_c2 as customer_id")
orderItems = orderItems.selectExpr("_c1 as order_id", "_c4 as order_item_subtotal")
customers = customers.selectExpr("_c0 as customer_id", "_c1 as customer_fname", "_c2 as customer_lname", "_c6 as customer_city")

#run spark sql queries
orderItems.createOrReplaceTempView("orderItems")
orderItemsFiltered = spark.sql("select order_id, sum(order_item_subtotal) as order_amount from orderItems group by order_id having sum(order_item_subtotal)>200")

#join dataframes
o_oi_join = orders.join(orderItemsFiltered, "order_id")
result = o_oi_join.join(customers, "customer_id")
final = result.selectExpr("customer_fname as fname", "customer_lname as lname", "customer_city as city", "order_amount as price")

#save to HDFS
final.write.option("sep",",").format("csv").save("/user/cloudera/practice2/problem3/joinResults")

### Practice 3- Question 2

`Instructions`
- Change permissions of all the files under /user/cloudera/problem3/customer/permissions such that owner has read,write and execute permissions, group has read and write permissions whereas others have just read and execute permissions

In [ ]:
sqoop import --connect "jdbc:mysql://localhost/retail_db" --password admin --username root --table customers --target-dir /user/cloudera/problem3/customer/permissions

hdfs dfs -chmod 765 /user/cloudera/problem3/customer/permissions/*

### Practice 3 - Question 3

`Instructions`
- Get count of customers in each city who have placed order of amount more than 100 and  whose order status is not PENDING.
- Input files are tab delimeted files placed at below HDFS location:
	- /user/cloudera/practice3/problem3/customers
	- /user/cloudera/practice3/problem3/orders
	- /user/cloudera/practice3/problem3/order_items

`Output Requirement`
- Output should be placed in below HDFS Location /user/cloudera/practice3/problem3/joinResults
- Output file should be tab separated file with deflate compression.

In [ ]:
#load data into HDFS
sqoop import --connect "jdbc:mysql://localhost/retail_db" --password admin --username root --table orders --fields-terminated-by "\t" --target-dir /user/cloudera/practice3/problem3/orders
sqoop import --connect "jdbc:mysql://localhost/retail_db" --password admin --username root --table order_items --fields-terminated-by "\t" --target-dir /user/cloudera/practice3/problem3/order_items
sqoop import --connect "jdbc:mysql://localhost/retail_db" --password admin --username root --table customers --fields-terminated-by "\t" --target-dir /user/cloudera/practice3/problem3/customers
 
#load dataset
orders = spark.read.option("sep","\t").csv("/user/cloudera/practice3/problem3/orders")
orderItems = spark.read.option("sep","\t").csv("/user/cloudera/practice3/problem3/order_items")
customers = spark.read.option("sep","\t").csv("/user/cloudera/practice3/problem3/customers")

#rename columns and filter data
ordersRenamed = orders.selectExpr("_c0 as order_id", "_c2 as customer_id", "_c3 as order_status")
orderItemsRenamed = orderItems.selectExpr("_c0 as order_item_id", "_c1 as order_id", "_c4 as order_item_subtotal")
customersRenamed = customers.selectExpr("_c0 as customer_id","_c6 as customer_city")
ordersF = ordersRenamed.filter(~ ordersRenamed.order_status.like("%PENDING%"))

#join dataframes and run spark sql queries
o_oi_join = ordersF.join(orderItemsRenamed, "order_id")
o_oi_join.createOrReplaceTempView("joins")
joinsF = spark.sql("select customer_id, sum(order_item_subtotal) as order_amount from joins group by customer_id having sum(order_item_subtotal) > 100")

result = joinsF.join(customersRenamed,"customer_id")
result.createOrReplaceTempView("res")
final = spark.sql("select customer_city, count(*) as count from res group by customer_city")

#save to Hive
final.write.option("sep","\t").option("compression","deflate").format("csv").save("/user/cloudera/practice3/problem3/joinResults")

### Practice 3 - Question 4

`Instructions`
- Input file is provided at below HDFS Location /user/cloudera/problem4_ques6/input
- Save the data to hdfs using no compression as sequence file.


`Output Requirement`
- Result should be saved in at /user/cloudera/problem4_ques6/output
- fields should be seperated by pipe delimiter.
- Key should be order_id, value should be all fields seperated by a pipe.

In [ ]:
#load data into HDFS
sqoop import --connect "jdbc:mysql://localhost/retail_db" --password cloudera --username root --table orders --target-dir /user/cloudera/problem4_ques6/input --as-parquetfile

#Read dataframe
orders = spark.read.format("parquet").load("/user/cloudera/problem4_ques6/input")

#add pipeline delimiters
ordersMap = orders.rdd.map(lambda line: "|".join([str(x) for x in line]))

#save to HDFS
ordersMap.map(lambda x: (None,x)).saveAsSequenceFile("/user/cloudera/problem4_ques6/output") 

### Practice 3 - Question5

`Instructions`
- using product_ranked_new metastore table, Find the top 5 most expensive products within each category 


`Output Requirement`
- Output should have product_category_id,product_name,product_price.
- Result should be saved in /user/cloudera/pratice4/question2/output
- Output should be saved as pipe delimited file.

In [ ]:
#load data into Hive
sqoop import --connect "jdbc:mysql://localhost/retail_db" --username root --password cloudera --table products --warehouse-dir /user/cloudera/practice5.db --hive-import --create-hive-table --hive-database default --hive-table product_ranked_new -m 1

#run spark sql queries
TopFiveProducts = spark.sql("product_category_id, product_name, product_price, dense_rank() over (partition by product_category_id order by product_price desc) as rank from product_ranked_new")

#filter dataframe
TopFiveProductsFiltered = TopFiveProducts.filter(TopFiveProducts.rank < 6).drop("rank")

#add pipeline delimiters
result = TopFiveProductsFiltered.map(lambda line: "|".join([str(x) for x in line]))

#save to HDFS
result.write.format("text").save("/user/cloudera/practice4/question2/output")


### Practice 3- Question6

`Instructions`
- Input file is provided at below HDFS location /user/cloudera/prac3/ques6/xml
- Append first three character of firstname with first two character of last name with a colon.


`Output Requirement`
- Output should be saved in xml file with rootTag as persons and rowTag as person.
- Output should be saved at below HDFS location /user/cloudera/prac3/ques6/output/xml

In [ ]:
#load data into HDFS
sqoop import --connect "jdbc:mysql://localhost/retail_db" --password admin --username root --table customers  --target-dir /user/cloudera/prac3/ques6/xml

#read and rename dataframe
customers = spark.read.format("csv").load("/user/cloudera/prac3/ques6/xml")
customersRN = customers.selectExpr("_c1 as fname", "_c2 as lname")

#run spark SQL query
customersRN.createOrReplaceTempView("res")

result = spark.sql("select concat(fname,':',lname) as fullname from res")

#save to HDFS
result.write.format("com.databricks.spark.xml").option("rootTag","persons").option("rowTag","persons").save("/user/cloudera/prac3/ques6/output/xml")
